In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**************Co -Ordinator*********************

why we use co-ordinator?:

agents are domain expert  where each agent reads contract context, what only what is needed in a json structure manner

Whereas ,co-ordinator decides what to show to user (the things asked by user only is displayed to the user co-ordinator is responsible for doing such things here)


In [ ]:
#Load Existing Agent Outputs
import json

def load_agent_output(path):
    with open(path, "r") as f:
        data = json.load(f)
    return data["output"]  # 🔥 ONLY this matters


In [ ]:
legal_output = load_agent_output("/content/drive/MyDrive/agents/legal_agent_output.json")
compliance_output = load_agent_output("/content/drive/MyDrive/agents/compliance_agent_output.json")
finance_output = load_agent_output("/content/drive/MyDrive/agents/finance_agent_output.json")
operations_output = load_agent_output("/content/drive/MyDrive/agents/operations_agent_output.json")


In [ ]:
#Define Coordinator Routing Rules
ROUTING_RULES = {
    "legal": ["termination", "governing law", "jurisdiction", "indemnity", "breach"],
    "compliance": ["gdpr", "audit", "regulatory", "data protection"],
    "finance": ["payment", "fee", "penalty", "invoice"],
    "operations": ["deliverable", "timeline", "sla", "milestone"]
}


here, keywords are very important.  Because , this is the main thing that where agent understand that this is related to something like(legal,compliance etc)

these keywords are used to map the agent like which agent  should come into play

In [ ]:
#Define Routing Function
def route_query(query: str):
    query = query.lower()
    selected_agents = []

    for agent, keywords in ROUTING_RULES.items():
        if any(keyword in query for keyword in keywords):
            selected_agents.append(agent)

    return selected_agents


core logic to map the keywords with the agent and returns list of agents used for that given query

In [ ]:
#Coordinator Execution Logic
def coordinator_execute(query: str):
    agents = route_query(query)
    results = {}

    for agent in agents:
        if agent == "legal":
            results["legal"] = legal_output
        elif agent == "compliance":
            results["compliance"] = compliance_output
        elif agent == "finance":
            results["finance"] = finance_output
        elif agent == "operations":
            results["operations"] = operations_output

    return {
        "query": query,
        "agents_used": agents,
        "results": results
    }


this is the main part of the system : the process is -

1.User query comes in

2.Routing decides which agents matter

3.Coordinator fetches existing outputs

4.Aggregates them

5.Returns a clean response


here , it won't rerun the agents and won't hallucinate

In [ ]:
query = "Explain termination and indemnity risks in the contract"
final_output = coordinator_execute(query)
print(json.dumps(final_output, indent=2))

{
  "query": "Explain termination and indemnity risks in the contract",
  "agents_used": [
    "legal"
  ],
  "results": {
    "legal": {
      "clause_type": "Legal Analysis",
      "extracted_clauses": [
        "The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
        "The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination."
      ],
      "risk_level": "low",
      "confidence": 0.85,
      "evidence": [
        "The first sentence of the clause describes different events that can lead to termination."
      ]
    }
  }
}


ClauseAI first uses a Retrieval-Augmented Generation (RAG) pipeline to retrieve pertinent contract sections. Only these retrieved sections are then examined by specialized agents, who then extract domain-specific clauses—like indemnity or termination—in a structured JSON format. To guarantee accuracy, every extracted clause is copied exactly from the contract. User queries are routed to the relevant agents by a coordinator layer, which then compiles their verified outputs into a single response. All extracted insights are guaranteed to be modular, explicable, and traceable back to the original contract text thanks to this layered approach.

Are Agents and co-ordinator are same ?

* agents are domain specific , and they will analyze what is going on etc..
* co-ordinator are cross domain ,rule based , just they are used to to decide which agents is used for what query and summarize all the agent't output into one that human can understand

why we need coordinator in this project?

Now

- Clean output

- Correct relevance

- Demo-ready

- Explainable

Future

- Scale to 20+ agents

- Plug into chat UI

- Enterprise workflows

- Replace routing logic without touching agents

 ********************Lang Graph***************

In [ ]:
pip install langgraph


what is Langgraph : Basically it is a framwork that lets us to:
* Represent agent's as node
* Represent execution order as edges
* maintian a shared state that flows through agents



Why we use langgraph in our project:
  * strctured execution
  * explicit graph
  * easy to add /remove agents
  * clear execution path


In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:
#Define Shared Graph State
from typing import TypedDict, Dict, Any
class ClauseAIState(TypedDict):
    query: str
    legal: Dict[str, Any]
    compliance: Dict[str, Any]
    finance: Dict[str, Any]
    operations: Dict[str, Any]


here,we defined a shared graph state that carries query and accumulated outputs from all agents as the graph executes

In [ ]:
import json

def load_output(path: str):
    with open(path, "r") as f:
        return json.load(f)["output"]


here, it will recieve shared graph state , perform one agent's work , update it to the shared graph state and finally return shared graph state

In [ ]:
LEGAL_KEYWORDS = ["termination", "indemnity", "governing law", "jurisdiction", "breach"]
COMPLIANCE_KEYWORDS = ["gdpr", "audit", "regulatory", "data protection"]
FINANCE_KEYWORDS = ["payment", "fee", "penalty", "invoice"]
OPERATIONS_KEYWORDS = ["sla", "timeline", "milestone", "deliverable"]


In [ ]:
##Legal Agent node

def legal_node(state: ClauseAIState) -> ClauseAIState:
    print("▶️ Executing Legal Agent")

    legal_output = load_output("/content/drive/MyDrive/agents/legal_agent_output.json")
    state["legal"] = legal_output

    return state


In [ ]:
## compliance Agent node
def compliance_node(state: ClauseAIState) -> ClauseAIState:
    print("▶️ Executing Compliance Agent")

    compliance_output = load_output("/content/drive/MyDrive/agents/compliance_agent_output.json")
    state["compliance"] = compliance_output

    return state


In [ ]:
## finance Agent node
def finance_node(state: ClauseAIState) -> ClauseAIState:
    print("▶️ Executing Finance Agent")

    finance_output = load_output("/content/drive/MyDrive/agents/finance_agent_output.json")
    state["finance"] = finance_output

    return state


In [ ]:
## operation Agent node
def operations_node(state: ClauseAIState) -> ClauseAIState:
    print("▶️ Executing Operations Agent")

    operations_output = load_output("/content/drive/MyDrive/agents/operations_agent_output.json")
    state["operations"] = operations_output

    return state



In [ ]:
#Build Graph Skeleton
from langgraph.graph import StateGraph
graph = StateGraph(ClauseAIState)


In [ ]:
#added nodes to the graph
graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)

In [ ]:
#Define Execution Flow
#Compliance → Legal → Finance → Operations
graph.set_entry_point("legal_agent")
graph.add_edge("legal_agent", "compliance_agent")
graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "operations_agent")
graph.add_edge("operations_agent", END)


Basic agents flow "Legal → Compliance → Finance → Operations → END
"

In [ ]:
app = graph.compile()

In [ ]:
input_state = {
    "query": "Explain termination and indemnity risks",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
res = app.invoke(input_state)
res.keys()

▶️ Executing Legal Agent
▶️ Executing Compliance Agent
▶️ Executing Finance Agent
▶️ Executing Operations Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
res['compliance']

{'clause_type': 'Compliance Analysis',
 'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."],
 'risk_level': 'high',
 'confidence': 1.0,
 'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]}

In [ ]:
input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
result = app.invoke(input_state)
result.keys()


▶️ Executing Legal Agent
▶️ Executing Compliance Agent
▶️ Executing Finance Agent
▶️ Executing Operations Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
#TEST CASES(LEGAL)
result['legal']

{'clause_type': 'Legal Analysis',
 'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
  'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'],
 'risk_level': 'low',
 'confidence': 0.85,
 'evidence': ['The first sentence of the clause describes different events that can lead to termination.']}

In [ ]:
#TEST CASE-2
result['compliance']

{'clause_type': 'Compliance Analysis',
 'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."],
 'risk_level': 'high',
 'confidence': 1.0,
 'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]}

In [ ]:
result['finance']

{'clause_type': 'Finance Analysis',
 'extracted_clauses': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ',
  '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.',
  'Provider shall provide Recipient with monthly invo'],
 'risk_level': 'medium',
 'confidence': 0.7,
 'evidence': ['The clause clearly defines the scope of Out-of-Pocket Costs and requires Recipient to reimburse Provider for such expenses.']}

In [ ]:
result['operations']

{'clause_type': 'Operations Analysis',
 'extracted_clauses': ['Collectible Concepts Group will obtain any licenses deemed by the Joint Venturers to add value in the marketing of the Products',
  'Pivotal Self Service Tech, Inc. will provide fulfillment services through affiliates for final distribution of the Products'],
 'risk_level': 'medium',
 'confidence': 0.8,
 'evidence': ['The clause specifies that Collectible Concepts Group will obtain any licenses deemed by the Joint Venturers to add value in the marketing of the Products',
  'The clause also specifies that Pivotal Self Service Tech, Inc. will provide fulfillment services through affiliates for final distribution of the Products']}

changing the agent's order to "compliance->Legal  → Finance → Operations → END"

In [ ]:
# 1️⃣ Create NEW graph
graph = StateGraph(ClauseAIState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")


graph.add_edge("compliance_agent", "legal_agent")
graph.add_edge("legal_agent", "finance_agent")
graph.add_edge("finance_agent", "operations_agent")
graph.add_edge("operations_agent", END)

app = graph.compile()


In [ ]:
input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
result = app.invoke(input_state)
result.keys()


▶️ Executing Compliance Agent
▶️ Executing Legal Agent
▶️ Executing Finance Agent
▶️ Executing Operations Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

when we changing the agent's order the graph is exactly working as expected
for example , entry agent is compliance here . So,Compliance agent is executing first

********************************************

********************************************
Removing finance agent

In [ ]:
# 1️⃣ Create NEW graph
graph = StateGraph(ClauseAIState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")


graph.add_edge("compliance_agent", "legal_agent")
graph.add_edge("legal_agent", "operations_agent")
graph.add_edge("operations_agent", END)

app = graph.compile()


In [ ]:
input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
result = app.invoke(input_state)
result.keys()


▶️ Executing Compliance Agent
▶️ Executing Legal Agent
▶️ Executing Operations Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
result['finance']

{}

*  Langgraph doent require all agents
* Removing an agent does not break the system

* Shared state remains consistent

* Pipeline adapts structurally

In [ ]:
import json
print(json.dumps(result, indent=2))


{
  "query": "Review termination, GDPR compliance, payment terms, and SLAs",
  "legal": {
    "clause_type": "Legal Analysis",
    "extracted_clauses": [
      "The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
      "The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination."
    ],
    "risk_level": "low",
    "confidence": 0.85,
    "evidence": [
      "The first sentence of the clause describes different events that can lead to termination."
    ]
  },
  "compliance": {
    "clause_type": "Compliance Analysis",
    "extracted_clauses": [
      "The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."
    ],
    "risk_level": "high",
    "confidence": 1.0,
    "evidence": [
      "The claus

# ******************Conditional Routing in LangGraph***********

In [ ]:
from typing import TypedDict, Dict, Any
class MultiAgentState(TypedDict):
    query: str
    legal: Dict[str, Any]
    compliance: Dict[str, Any]
    finance: Dict[str, Any]
    operations: Dict[str, Any]


In [ ]:
#define the route fun
ROUTING_KEYWORDS = {
    "legal_agent": [
        "termination", "indemnity", "governing law", "jurisdiction", "breach"
    ],
    "compliance_agent": [
        "gdpr", "audit", "regulatory", "data protection"
    ],
    "finance_agent": [
        "payment", "fee", "penalty", "invoice", "late payment"
    ],
    "operations_agent": [
        "sla", "timeline", "milestone", "deliverable"
    ]
}


In [ ]:
def router_node(state: MultiAgentState) -> MultiAgentState:
    print("🔀 Router node executed")
    return state


In [ ]:
def route_query(state: MultiAgentState) -> str:
    query = state["query"].lower()

    for agent, keywords in ROUTING_KEYWORDS.items():
        if any(k in query for k in keywords):
            print(f"🔀 Routing to {agent}")
            return agent

    print("🔀 No match → defaulting to legal_agent")
    return "legal_agent"


we are changing the path to

START → Router

              ├── Legal → END

              ├── Compliance → END

              ├── Finance → END
              
              └── Operations → END


In [ ]:
#build a graph with conditional entry
from langgraph.graph import StateGraph
graph = StateGraph(MultiAgentState)


In [ ]:
#added the nodes
graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)


graph.add_node("router", router_node)


here, router is decision node that returns the name of the next node and langgraph follows that decision

In [ ]:
graph.set_entry_point("router") #we are saying langgraph that execution should start at the router

In [ ]:
from langgraph.graph import END
graph.add_edge("legal_agent", END)
graph.add_edge("compliance_agent", END)
graph.add_edge("finance_agent", END)
graph.add_edge("operations_agent", END)


here, we saying that once that selected agent finishes the execution ,stop the graph  because without this execution is incomplete and langgraph will throw errors

In [ ]:
graph.add_conditional_edges(
    "router",
    route_query,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
    }
)


We now connect the router to agents conditionally.

This tells LangGraph:
* “Based on what the router returns, go to the corresponding agent node.”

In [ ]:
app = graph.compile()


In [ ]:
#TEST CASES
state = {
    "query": "Review termination clause",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}


In [ ]:
result = app.invoke(state)
result.keys()

🔀 Router node executed
🔀 Routing to legal_agent
▶️ Executing Legal Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
result['legal']

{'clause_type': 'Legal Analysis',
 'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
  'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'],
 'risk_level': 'low',
 'confidence': 0.85,
 'evidence': ['The first sentence of the clause describes different events that can lead to termination.']}

In [ ]:
state = {
"query": "Check late payment penalties",
"legal": {},
"compliance": {},
"finance": {},
"operations": {}
}

result = app.invoke(state)
result.keys()

🔀 Router node executed
🔀 Routing to finance_agent
▶️ Executing Finance Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
result['finance']

{'clause_type': 'Finance Analysis',
 'extracted_clauses': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ',
  '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.',
  'Provider shall provide Recipient with monthly invo'],
 'risk_level': 'medium',
 'confidence': 0.7,
 'evidence': ['The clause clearly defines the scope of Out-of-Pocket Costs and requires Recipient to reimburse Provider for such expenses.']}

******************TEST CASE 3 — MULTIPLE INTENT

In [ ]:
from typing import TypedDict, Dict, Any
from langgraph.graph import StateGraph, END
import json


In [ ]:
class ClauseAIMultiAgentState(TypedDict):
    query: str
    legal: Dict[str, Any]
    compliance: Dict[str, Any]
    finance: Dict[str, Any]
    operations: Dict[str, Any]


In [ ]:
ROUTING_KEYWORDS = {
    "legal": ["termination", "governing law", "jurisdiction", "indemnity"],
    "compliance": ["gdpr", "audit", "regulatory", "data protection"],
    "finance": ["payment", "fee", "penalty", "invoice"],
    "operations": ["deliverable", "timeline", "sla", "milestone"]
}


In [ ]:
def route_query_multi(state: ClauseAIMultiAgentState):
    query = state["query"].lower()
    selected_agents = []

    for agent, keywords in ROUTING_KEYWORDS.items():
        if any(keyword in query for keyword in keywords):
            selected_agents.append(agent)

    if not selected_agents:
        selected_agents = ["legal"]  # fallback

    print(f"🔀 Selected agents: {selected_agents}")
    return selected_agents


In [ ]:
def load_output(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)["output"]




In [ ]:
def legal_node(state: ClauseAIMultiAgentState):
    print("▶️ Executing Legal Agent")
    state["legal"] = load_output(
        "/content/drive/MyDrive/agents/legal_agent_output.json"
    )
    return state


def compliance_node(state: ClauseAIMultiAgentState):
    print("▶️ Executing Compliance Agent")
    state["compliance"] = load_output(
        "/content/drive/MyDrive/agents/compliance_agent_output.json"
    )
    return state


def finance_node(state: ClauseAIMultiAgentState):
    print("▶️ Executing Finance Agent")
    state["finance"] = load_output(
        "/content/drive/MyDrive/agents/finance_agent_output.json"
    )
    return state


def operations_node(state: ClauseAIMultiAgentState):
    print("▶️ Executing Operations Agent")
    state["operations"] = load_output(
        "/content/drive/MyDrive/agents/operations_agent_output.json"
    )
    return state


In [ ]:
def coordinator_node(state: ClauseAIMultiAgentState):
    agents = route_query_multi(state)

    for agent in agents:
        if agent == "legal":
            state = legal_node(state)
        elif agent == "compliance":
            state = compliance_node(state)
        elif agent == "finance":
            state = finance_node(state)
        elif agent == "operations":
            state = operations_node(state)

    return state


In [ ]:
graph = StateGraph(ClauseAIMultiAgentState)
graph.add_node("coordinator", coordinator_node)
graph.set_entry_point("coordinator")
graph.add_edge("coordinator", END)

app = graph.compile()


In [ ]:
state = {
    "query": "Check GDPR compliance and payment terms",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}


In [ ]:
result = app.invoke(state)
result.keys()


🔀 Selected agents: ['compliance', 'finance']
▶️ Executing Compliance Agent
▶️ Executing Finance Agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
result['compliance']

{'clause_type': 'Compliance Analysis',
 'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."],
 'risk_level': 'high',
 'confidence': 1.0,
 'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]}

In [ ]:
result['finance']

{'clause_type': 'Finance Analysis',
 'extracted_clauses': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ',
  '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.',
  'Provider shall provide Recipient with monthly invo'],
 'risk_level': 'medium',
 'confidence': 0.7,
 'evidence': ['The clause clearly defines the scope of Out-of-Pocket Costs and requires Recipient to reimburse Provider for such expenses.']}

In [ ]:
result['operations']

{}

extra Task :


```
- Add new keyword mapping
- Test multiple queries
- Observe which agent is selected
```

In [ ]:
ROUTING_KEYWORDS = {
    "legal": ["termination","governing law","jurisdiction","indemnity","liability","dispute"],
    "compliance": ["gdpr","audit","regulatory","data protection","privacy"],
    "finance": ["payment","fee","penalty","invoice","late payment"],
    "operations": ["deliverable","timeline","sla","milestone","service level"]
}

In [ ]:
state = {
    "query": "Explain indemnity and liability clauses",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)


🔀 Selected agents: ['legal']
▶️ Executing Legal Agent


In [ ]:
state = {
    "query": "Check GDPR and data protection compliance",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)


🔀 Selected agents: ['compliance']
▶️ Executing Compliance Agent


In [ ]:
state = {
    "query": "Review SLAs and milestone timelines",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)


🔀 Selected agents: ['operations']
▶️ Executing Operations Agent


In [ ]:
state = {
    "query": "Check GDPR compliance and payment terms and termination clause",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)


🔀 Selected agents: ['legal', 'compliance', 'finance']
▶️ Executing Legal Agent
▶️ Executing Compliance Agent
▶️ Executing Finance Agent


# **Conversation Memory & State Persistence**

Without memory:

- Agents act independently

- No awareness of previous findings

- No cross-agent reasoning

With memory:

- Agents can see what others found

- You can build:

    - Refinement

    - Validation

    - Summarization

    - Conflict detection

In [ ]:
#Update Graph State (Add Memory Field)
from typing import TypedDict, List, Dict, Any

class GraphState(TypedDict):
    query: str
    memory: List[dict]
    legal: Dict[str, Any]
    compliance: Dict[str, Any]
    finance: Dict[str, Any]
    operations: Dict[str, Any]


In [ ]:
#intialize the memory in input state
input_state = {
    "query": "Check GDPR compliance and payment terms",
    "memory": [],
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}


here,memory should be empty at starting stage becuase later every agent will add one record later in below format

{
    "agent": "legal",
    "output": {...}
}


In [ ]:
#Modify Agent Nodes to Write to Memory
def legal_node(state: GraphState):
    print("▶️ Executing Legal Agent")

    legal_output = load_output(
        "/content/drive/MyDrive/agents/legal_agent_output.json"
    )

    # Update agent-specific state
    state["legal"] = legal_output
    # 🔹 Write to memory
    state["memory"].append({
        "agent": "legal",
        "output": legal_output
    })
    return state


def compliance_node(state: GraphState):
    print("▶️ Executing compliance Agent")

    compliance_output = load_output(
        "/content/drive/MyDrive/agents/compliance_agent_output.json"
    )

    # Update agent-specific state
    state["compliance"] = compliance_output
    # 🔹 Write to memory
    state["memory"].append({
        "agent": "compliance",
        "output": compliance_output
    })
    return state



def finance_node(state: GraphState):
    print("▶️ Executing finance Agent")

    finance_output = load_output(
        "/content/drive/MyDrive/agents/finance_agent_output.json"
    )

    # Update agent-specific state
    state["finance"] = finance_output
    # 🔹 Write to memory
    state["memory"].append({
        "agent": "finance",
        "output": finance_output
    })
    return state


def operations_node(state: GraphState):
    print("▶️ Executing operations Agent")

    operations_output = load_output(
        "/content/drive/MyDrive/agents/operations_agent_output.json"
    )

    # Update agent-specific state
    state["operations"] = operations_output
    # 🔹 Write to memory
    state["memory"].append({
        "agent": "operations",
        "output": operations_output
    })
    return state

In [ ]:
def coordinator_node(state: GraphState):
    agents = route_query_multi(state)

    for agent in agents:
        if agent == "legal":
            state = legal_node(state)

        elif agent == "compliance":
            state = compliance_node(state)

        elif agent == "finance":
            state = finance_node(state)

        elif agent == "operations":
            state = operations_node(state)

    return state


Co-Ordinator Node is the entry point of the graph it decides which agents run and it passes state(including memory) forward

In [ ]:
def route_query_multi(state: GraphState):
    query = state["query"].lower()
    selected_agents = []

    for agent, keywords in ROUTING_KEYWORDS.items():
        if any(k in query for k in keywords):
            selected_agents.append(agent)

    if not selected_agents:
        selected_agents = ["legal"]

    print(f"🔀 Selected agents: {selected_agents}")
    return selected_agents


In [ ]:
import json

def load_output(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)["output"]


In [ ]:
graph = StateGraph(GraphState)

graph.add_node("coordinator", coordinator_node)

graph.set_entry_point("coordinator")
graph.add_edge("coordinator", END)

app = graph.compile()


In [ ]:
app = graph.compile()


In [ ]:
input_state = {
    "query": "Check GDPR compliance and payment terms",
    "memory": [],
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
result = app.invoke(input_state)

🔀 Selected agents: ['compliance', 'finance']
▶️ Executing compliance Agent
▶️ Executing finance Agent


In [ ]:
result['compliance']

{'clause_type': 'Compliance Analysis',
 'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."],
 'risk_level': 'high',
 'confidence': 1.0,
 'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]}

In [ ]:
result['finance']

{'clause_type': 'Finance Analysis',
 'extracted_clauses': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ',
  '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.',
  'Provider shall provide Recipient with monthly invo'],
 'risk_level': 'medium',
 'confidence': 0.7,
 'evidence': ['The clause clearly defines the scope of Out-of-Pocket Costs and requires Recipient to reimburse Provider for such expenses.']}

In [ ]:
result['legal']

{}

In [ ]:
print("🧠 Memory Contents:\n")
for i, entry in enumerate(result["memory"], start=1):
    print(f"Step {i}:")
    print(f"Agent: {entry['agent']}")
    print(f"Output keys: {list(entry['output'].keys())}")
    print("-" * 40)


🧠 Memory Contents:

Step 1:
Agent: compliance
Output keys: ['clause_type', 'extracted_clauses', 'risk_level', 'confidence', 'evidence']
----------------------------------------
Step 2:
Agent: finance
Output keys: ['clause_type', 'extracted_clauses', 'risk_level', 'confidence', 'evidence']
----------------------------------------


In [ ]:
import pprint
pprint.pprint(result["memory"])


[{'agent': 'compliance',
  'output': {'clause_type': 'Compliance Analysis',
             'confidence': 1.0,
             'evidence': ['The clause clearly requires the receiving party to '
                          "maintain confidentiality of the other party's "
                          'confidential information.'],
             'extracted_clauses': ['The receiving party will not disclose the '
                                   "other party's confidential information to "
                                   'any third parties without the other '
                                   "party's prior written consent."],
             'risk_level': 'high'}},
 {'agent': 'finance',
  'output': {'clause_type': 'Finance Analysis',
             'confidence': 0.7,
             'evidence': ['The clause clearly defines the scope of '
                          'Out-of-Pocket Costs and requires Recipient to '
                          'reimburse Provider for such expenses.'],
             'extracted_cl

her , we got correct thing like , memory is started with empty list and now it contains executed agent's output

In [ ]:
def legal_node(state: GraphState):
    legal_output = load_output(
        "/content/drive/MyDrive/agents/legal_agent_output.json"
    )

    # Store latest legal output
    state["legal"] = legal_output

    # Log to memory
    state["memory"].append({
        "agent": "legal",
        "output": legal_output
    })

    return state


In [ ]:
def coordinator_node(state: GraphState):
    agents = route_query_multi(state)

    for agent in agents:
        if agent == "legal":
            state = legal_node(state)

        elif agent == "compliance":
            state = compliance_node(state)

        elif agent == "finance":
            state = finance_node(state)

        elif agent == "operations":
            state = operations_node(state)

        # 🔍 PRINT MEMORY AFTER EACH AGENT
        print("🧠 Memory so far:")
        for i, m in enumerate(state["memory"], start=1):
            print(f"  {i}. {m['agent']}")
        print("-" * 40)

    return state


In [ ]:
#----------------agent to agent communication----

In [ ]:
#update graph state
from typing import TypedDict, List, Dict, Any

class GraphState(TypedDict):
    query: str
    memory: List[Dict[str, Any]]
    validation_notes: List[str]
    compliance: Dict[str, Any]
    finance: Dict[str, Any]
    legal: Dict[str, Any]
    operations: Dict[str, Any]


In [ ]:
#Initialize State (Multi-Intent Query)
input_state: GraphState = {
    "query": "Review SLA enforceability, compliance risks, and financial penalties",
    "memory": [],
    "validation_notes": [],
    "compliance": {},
    "finance": {},
    "legal": {},
    "operations": {}
}


In [ ]:
#compilence agent
def compliance_node(state: GraphState):
    output = {
        "extracted_clauses": [
            "Data protection obligations",
            "Regulatory audit rights"
        ]
    }

    state["compliance"] = output

    state["memory"].append({
        "agent": "compliance",
        "findings": output["extracted_clauses"]
    })

    print("\n📘 After Compliance Agent Memory:")
    for m in state["memory"]:
        print(m)

    return state


In [ ]:
#Finance Agent (Reads Compliance Memory + Validates)
def finance_node(state: GraphState):
    compliance_findings = [
        m for m in state["memory"] if m["agent"] == "compliance"
    ]

    output = {
        "extracted_clauses": [
            "Penalty cap",
            "Late payment interest"
        ]
    }

    state["finance"] = output

    if compliance_findings:
        state["validation_notes"].append(
            "Finance reviewed compliance findings for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": output["extracted_clauses"]
    })

    print("\n💰 After Finance Agent Memory:")
    for m in state["memory"]:
        print(m)

    return state


In [ ]:
#Legal Agent (Final Validation Layer)
def legal_node(state: GraphState):
    finance_findings = [
        m for m in state["memory"] if m["agent"] == "finance"
    ]

    output = {
        "extracted_clauses": [
            "Limitation of liability",
            "Governing law",
            "SLA enforceability"
        ]
    }

    state["legal"] = output

    if finance_findings:
        state["validation_notes"].append(
            "Legal validated financial penalties against liability framework."
        )

    state["memory"].append({
        "agent": "legal",
        "findings": output["extracted_clauses"]
    })

    print("\n⚖️ After Legal Agent Memory:")
    for m in state["memory"]:
        print(m)

    return state


In [ ]:
#Operations Agent (Reads Legal Output + SLA Validation
def operations_node(state: GraphState):
    legal_findings = [
        m for m in state["memory"] if m["agent"] == "legal"
    ]

    output = {
        "extracted_clauses": [
            "Service uptime SLA",
            "Incident response timelines"
        ]
    }

    state["operations"] = output

    if legal_findings:
        state["validation_notes"].append(
            "Operations validated SLA enforceability against legal terms."
        )

    state["memory"].append({
        "agent": "operations",
        "findings": output["extracted_clauses"]
    })

    print("\n⚙️ After Operations Agent Memory:")
    for m in state["memory"]:
        print(m)

    return state


In [ ]:
#Build Collaborative Graph
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("legal_agent", legal_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")
graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "legal_agent")
graph.add_edge("legal_agent", "operations_agent")
graph.add_edge("operations_agent", END)


In [ ]:
#Compile & Execute Collaborative Flow
app = graph.compile()
final_state = app.invoke(input_state)



📘 After Compliance Agent Memory:
{'agent': 'compliance', 'findings': ['Data protection obligations', 'Regulatory audit rights']}

💰 After Finance Agent Memory:
{'agent': 'compliance', 'findings': ['Data protection obligations', 'Regulatory audit rights']}
{'agent': 'finance', 'findings': ['Penalty cap', 'Late payment interest']}

⚖️ After Legal Agent Memory:
{'agent': 'compliance', 'findings': ['Data protection obligations', 'Regulatory audit rights']}
{'agent': 'finance', 'findings': ['Penalty cap', 'Late payment interest']}
{'agent': 'legal', 'findings': ['Limitation of liability', 'Governing law', 'SLA enforceability']}

⚙️ After Operations Agent Memory:
{'agent': 'compliance', 'findings': ['Data protection obligations', 'Regulatory audit rights']}
{'agent': 'finance', 'findings': ['Penalty cap', 'Late payment interest']}
{'agent': 'legal', 'findings': ['Limitation of liability', 'Governing law', 'SLA enforceability']}
{'agent': 'operations', 'findings': ['Service uptime SLA', 'Inc

In [ ]:
#Inspect Final Memory & Validation Notes
print("\n🧠 Final Accumulated Memory:")
for idx, m in enumerate(final_state["memory"], start=1):
    print(f"{idx}. {m['agent']} → {m['findings']}")

print("\n✅ Validation Notes:")
for note in final_state["validation_notes"]:
    print("-", note)



🧠 Final Accumulated Memory:
1. compliance → ['Data protection obligations', 'Regulatory audit rights']
2. finance → ['Penalty cap', 'Late payment interest']
3. legal → ['Limitation of liability', 'Governing law', 'SLA enforceability']
4. operations → ['Service uptime SLA', 'Incident response timelines']

✅ Validation Notes:
- Finance reviewed compliance findings for penalty conflicts.
- Legal validated financial penalties against liability framework.
- Operations validated SLA enforceability against legal terms.


In [ ]:
#-----------------pipeline for all

In [ ]:
from typing import List, Dict


In [ ]:
#Mock RAG Retriever (replace with Pinecone / FAISS)
def retrieve_chunks(keywords: List[str]) -> List[str]:
    corpus = {
        "regulatory": "The vendor must comply with all applicable regulations.",
        "data protection": "Personal data shall be processed under GDPR standards.",
        "audit": "Customer may audit systems annually.",
        "penalty": "Penalties apply for SLA breaches.",
        "interest": "Late payments accrue interest at 12% per annum.",
        "liability": "Liability is capped at annual contract value.",
        "indemnification": "Vendor shall indemnify against third-party claims.",
        "uptime": "Service uptime must be 99.9% monthly."
    }

    return [
        text for key, text in corpus.items()
        if any(k.lower() in key.lower() for k in keywords)
    ]


In [ ]:
#Combine Retrieved Chunks
def combine_chunks(chunks: List[str]) -> str:
    return "\n".join(chunks)


In [ ]:
#Generic Agent Executor
def run_agent(context: str, domain: str) -> Dict:
    clauses = context.split("\n") if context else []

    risk = "LOW"
    if len(clauses) >= 3:
        risk = "MEDIUM"
    if len(clauses) >= 5:
        risk = "HIGH"

    return {
        "domain": domain,
        "extracted_clauses": clauses,
        "confidence_score": round(min(0.95, 0.6 + 0.1 * len(clauses)), 2),
        "risk_level": risk
    }


In [ ]:
#Compliance pipeline
#Compliance Query
COMPLIANCE_KEYWORDS = [
    "regulatory",
    "data protection",
    "audit",
    "reporting"   # 🔁 changed keyword
]


In [ ]:
#run compliance pipeline
def compliance_pipeline():
    chunks = retrieve_chunks(COMPLIANCE_KEYWORDS)
    context = combine_chunks(chunks)
    output = run_agent(context, "compliance")

    output["summary"] = f"{len(output['extracted_clauses'])} compliance clauses identified."

    return output


In [ ]:
#finance piipeline
FINANCE_KEYWORDS = [
    "penalty",
    "payment",
    "interest"   # ✅ added
]


In [ ]:
#run finance pipeline
def finance_pipeline():
    chunks = retrieve_chunks(FINANCE_KEYWORDS)
    context = combine_chunks(chunks)
    output = run_agent(context, "finance")

    output["summary"] = f"Financial exposure risk: {output['risk_level']}"

    return output


In [ ]:
#legal pipeline
#legal query
LEGAL_KEYWORDS = [
    "liability",
    "governing law",
    "indemnification"   # ✅ added
]


In [ ]:
def legal_pipeline():
    chunks = retrieve_chunks(LEGAL_KEYWORDS)
    context = combine_chunks(chunks)
    output = run_agent(context, "legal")

    output["summary"] = "Core legal protections extracted."

    return output


In [ ]:
#operatinal pipeline
OPERATIONS_KEYWORDS = [
    "delivery",
    "timeline",
    "uptime"   # ✅ added
]


In [ ]:
#run operational pipeline
def operations_pipeline():
    chunks = retrieve_chunks(OPERATIONS_KEYWORDS)
    context = combine_chunks(chunks)
    output = run_agent(context, "operations")

    output["summary"] = "Operational execution risks assessed."

    return output


In [ ]:
#excute all pipelines
pipelines = {
    "Compliance": compliance_pipeline(),
    "Finance": finance_pipeline(),
    "Legal": legal_pipeline(),
    "Operations": operations_pipeline()
}

for name, result in pipelines.items():
    print(f"\n🔹 {name} Pipeline Output")
    print("Clauses:", result["extracted_clauses"])
    print("Confidence:", result["confidence_score"])
    print("Risk Level:", result["risk_level"])



🔹 Compliance Pipeline Output
Clauses: ['The vendor must comply with all applicable regulations.', 'Personal data shall be processed under GDPR standards.', 'Customer may audit systems annually.']
Confidence: 0.9
Risk Level: MEDIUM

🔹 Finance Pipeline Output
Clauses: ['Penalties apply for SLA breaches.', 'Late payments accrue interest at 12% per annum.']
Confidence: 0.8
Risk Level: LOW

🔹 Legal Pipeline Output
Clauses: ['Liability is capped at annual contract value.', 'Vendor shall indemnify against third-party claims.']
Confidence: 0.8
Risk Level: LOW

🔹 Operations Pipeline Output
Clauses: ['Service uptime must be 99.9% monthly.']
Confidence: 0.7
Risk Level: LOW


In [ ]:
#--------Coordinator: Merging Agent Outputs----------

In [ ]:
# Run individual pipelines
compliance_pipeline_output = compliance_pipeline()
finance_pipeline_output = finance_pipeline()
legal_pipeline_output = legal_pipeline()
operations_pipeline_output = operations_pipeline()

# Verify
print("Compliance Output:", compliance_pipeline_output)
print("Finance Output:", finance_pipeline_output)
print("Legal Output:", legal_pipeline_output)
print("Operations Output:", operations_pipeline_output)


Compliance Output: {'domain': 'compliance', 'extracted_clauses': ['The vendor must comply with all applicable regulations.', 'Personal data shall be processed under GDPR standards.', 'Customer may audit systems annually.'], 'confidence_score': 0.9, 'risk_level': 'MEDIUM', 'summary': '3 compliance clauses identified.'}
Finance Output: {'domain': 'finance', 'extracted_clauses': ['Penalties apply for SLA breaches.', 'Late payments accrue interest at 12% per annum.'], 'confidence_score': 0.8, 'risk_level': 'LOW', 'summary': 'Financial exposure risk: LOW'}
Legal Output: {'domain': 'legal', 'extracted_clauses': ['Liability is capped at annual contract value.', 'Vendor shall indemnify against third-party claims.'], 'confidence_score': 0.8, 'risk_level': 'LOW', 'summary': 'Core legal protections extracted.'}
Operations Output: {'domain': 'operations', 'extracted_clauses': ['Service uptime must be 99.9% monthly.'], 'confidence_score': 0.7, 'risk_level': 'LOW', 'summary': 'Operational execution 

In [ ]:
merged_output = coordinator_merge(
    legal_pipeline_output,
    compliance_pipeline_output,
    finance_pipeline_output,
    operations_pipeline_output
)


In [ ]:
################

In [ ]:
#Input: Pipeline Outputs
legal_output = legal_pipeline_output
compliance_output = compliance_pipeline_output
finance_output = finance_pipeline_output
operations_output = operations_pipeline_output


In [ ]:
#Define Final Output Schema
from typing import Dict, Any


In [ ]:
final_output = {
    "domains": {},
    "overall_risk": "",
    "overall_confidence": 0.0,
    "highest_risk_clause": ""
}


In [ ]:
#Merge Pipeline Outputs (Coordinator)
def coordinator_merge(
    legal: Dict[str, Any],
    compliance: Dict[str, Any],
    finance: Dict[str, Any],
    operations: Dict[str, Any]
) -> Dict[str, Any]:

    merged = {
        "domains": {
            "legal": legal,
            "compliance": compliance,
            "finance": finance,
            "operations": operations
        }
    }

    return merged


In [ ]:
merged_output = coordinator_merge(
    legal_output,
    compliance_output,
    finance_output,
    operations_output
)

merged_output.keys()


dict_keys(['domains'])

In [ ]:
#Confidence Aggregation Logic
def aggregate_confidence(merged: Dict[str, Any]) -> float:     #We’ll compute average confidence across all pipelines.
    scores = [
        domain["confidence_score"]
        for domain in merged["domains"].values()
        if "confidence_score" in domain
    ]

    return round(sum(scores) / len(scores), 2) if scores else 0.0


In [ ]:
#Compute Overall Risk (Highest Wins)
RISK_PRIORITY = {
    "LOW": 1,
    "MEDIUM": 2,
    "HIGH": 3
}

def compute_overall_risk(merged: Dict[str, Any]) -> str:
    risks = [
        domain["risk_level"]
        for domain in merged["domains"].values()
        if "risk_level" in domain
    ]

    return max(risks, key=lambda r: RISK_PRIORITY.get(r, 0))


In [ ]:
#Print Highest-Risk Clause
def get_highest_risk_clause(merged: Dict[str, Any]) -> str:
    highest_domain = max(
        merged["domains"].values(),
        key=lambda d: RISK_PRIORITY.get(d["risk_level"], 0)
    )

    clauses = highest_domain.get("extracted_clauses", [])
    return clauses[0] if clauses else "No clause identified"


In [ ]:
#Build Final Unified JSON Output
final_json = {
    "domains": merged_output["domains"],
    "overall_risk": compute_overall_risk(merged_output),
    "overall_confidence": aggregate_confidence(merged_output),
    "highest_risk_clause": get_highest_risk_clause(merged_output)
}



In [ ]:
#Inspect Final Output
print("\n📦 FINAL COORDINATED OUTPUT\n")

print("Overall Risk Level:", final_json["overall_risk"])
print("Overall Confidence:", final_json["overall_confidence"])
print("Highest Risk Clause:", final_json["highest_risk_clause"])



📦 FINAL COORDINATED OUTPUT

Overall Risk Level: MEDIUM
Overall Confidence: 0.8
Highest Risk Clause: The vendor must comply with all applicable regulations.
